In [ ]:
%pip install -U finance-datareader

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import FinanceDataReader as fdr

In [ ]:
import re

In [ ]:
stocks = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 전체

In [ ]:
stocks

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,75900,2,-2400,-3.07,76800,77000,75700,27891278,2127292051700,453106495545000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,198600,2,-1400,-0.70,199800,202500,197500,3793872,757806938160,144581269689000,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,356000,2,-6000,-1.66,358500,359000,355500,213011,76007017500,83304000000000,234000000,STK
3,005380,KR7005380001,현대차,KOSPI,,267500,2,-4000,-1.47,268500,274500,266000,1059149,285205873000,56018831092500,209416191,STK
4,207940,KR7207940008,삼성바이오로직스,KOSPI,,751000,2,-19000,-2.47,763000,766000,751000,81016,61292964000,53451674000000,71174000,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2804,413300,KR7413300005,티엘엔지니어링,KONEX,일반기업부,2490,5,-435,-14.87,3220,3220,2490,262,669570,3250732350,1305515,KNX
2805,217320,KR7217320001,썬테크,KONEX,일반기업부,2600,1,195,8.11,2600,2600,2600,1,2600,2730000000,1050000,KNX
2806,245450,KR7245450002,씨앤에스링크,KONEX,일반기업부,1699,1,203,13.57,1272,1699,1272,547,706756,2684352040,1579960,KNX
2807,236030,KR7236030003,씨알푸드,KONEX,일반기업부,1200,1,135,12.68,1099,1200,1000,5003,5914077,2439998400,2033332,KNX


In [ ]:
stocks['Symbol'] = stocks['Code'].astype(str)
def remove_noise_and_split_title(title):
    in_code = ''
    in_name = ''

    for code, name in stocks[['Symbol', 'Name']].values:
        if code in title and name in title:
            in_code = code
            in_name = name

    # 한글, 영어, 숫자 외 노이즈 제거
    clean_title = re.sub('[^A-Za-z0-9가-힣]', ' ', title)

    # 기업명 코드 수정
    clean_title = clean_title.replace(in_code, ' ')
    clean_title = clean_title.replace(in_name, ' ')
    while ' ' * 2 in clean_title:
        clean_title = clean_title.replace(' ' * 2, ' ')

    if in_name == '': # 기업명이 없는 제목이라면, 데이터에 추가하지 않음
        return [None]
    else:
        return [in_name, in_code, clean_title]

In [ ]:
url = "http://consensus.hankyung.com/analysis/list?sdate=2023-05-24&edate=2024-05-24&now_page=1&search_value=&report_type=CO&pagenum=20&search_text=&business_code="

In [ ]:
html = requests.get(url, headers={'User-Agent':'Gils'}).content

In [ ]:
soup = BeautifulSoup(html, 'lxml')

In [ ]:
soup

<!DOCTYPE html>
<html lang="ko">
<head>
<title>한경 컨센서스</title>
<meta content="text/html;" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex" name="robots"/>
<meta content="noindex" name="googlebot"/>
<link href="/css/common.css?v=20171065" rel="stylesheet" type="text/css"/>
<link href="/js/datePicker/css/ui-lightness/jquery-ui-1.10.2.custom.css" rel="stylesheet" type="text/css"/>
<link href="/js/datePicker/css/ui.daterangepicker.css" rel="stylesheet" type="text/css"/>
<link href="https://www.hankyung.com/favicon.ico" rel="shortcut icon"/>
<script src="https://hkstatic.hankyung.com/plugin/jquery-1.12.4.min.js" type="text/javascript"></script>
<script src="/js/jquery-1.12.4.min.js" type="text/javascript"></script>
<script src="/js/common.js?v=20231221" type="text/javascript"></script>
<script src="/js/datePicker/jquery-calendar.js" type="text/javascript"></script>
<script src="/js/datePicker/daterangepicker.jQuery.js?v=20231221" typ

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
start_date = "2023-01-01"
end_date = "2024-03-31"

In [ ]:
base_url = "http://consensus.hankyung.com/analysis/list?&sdate=2000-01-01&edate=2009-12-31&report_type=CO&order_type=&now_page={}"
data = []

for page_no in range(1,4133):
    while True:
        try:
            url = base_url.format(page_no)
            html = requests.get(url, headers={'User-Agent':'Gils'}).content
            soup = BeautifulSoup(html, 'lxml')
            print("{}/{}".format(page_no, 4133))
            break
        except:
            time.sleep(15 * 60)

    table = soup.find("div", {"class":"table_style01"}).find('table')
    for tr in table.find_all("tr")[1:]: # 1번째 행부터 순회
        record = []
        for i, td in enumerate(tr.find_all("td")[:6]): # 6번째 셀까지 순회
            if i == 1:
                record += remove_noise_and_split_title(td.text) # remove_noise_title의 출력과 이어 붙임
            elif i == 3: # 노이즈가 껴있는 세번째 셀만 따로 처리
                record.append(td.text.replace(" ", "").replace("\r","").replace("\n",""))
            else: # 1번째 셀이 아니면:
                record.append(td.text) # 셀의 텍스트 값 그대로 입력

        if None not in record: # 레코드에 None이 없으면
            data.append(record)

    time.sleep(1) # 연결 끊김 방지를 위해, 1초씩 재움

data = pd.DataFrame(data, columns = ["작성일", "종목명", "종목코드", "제목", "적정가격", "평가의견", "작성자", "작성기관"])
data.to_csv("리포트_데이터.csv", index = False, encoding = "cp949")

1/4133
2/4133
3/4133
4/4133
5/4133
6/4133
7/4133
8/4133
9/4133
10/4133
11/4133
12/4133
13/4133
14/4133
15/4133
16/4133
17/4133
18/4133
19/4133
20/4133
21/4133
22/4133
23/4133
24/4133
25/4133
26/4133
27/4133
28/4133
29/4133
30/4133
31/4133
32/4133
33/4133
34/4133
35/4133
36/4133
37/4133
38/4133
39/4133
40/4133
41/4133
42/4133
43/4133
44/4133
45/4133
46/4133
47/4133
48/4133
49/4133
50/4133
51/4133
52/4133
53/4133
54/4133
55/4133
56/4133
57/4133
58/4133
59/4133
60/4133
61/4133
62/4133
63/4133
64/4133
65/4133
66/4133
67/4133
68/4133
69/4133
70/4133
71/4133
72/4133
73/4133
74/4133
75/4133
76/4133
77/4133
78/4133
79/4133
80/4133
81/4133
82/4133
83/4133
84/4133
85/4133
86/4133
87/4133
88/4133
89/4133
90/4133
91/4133
92/4133
93/4133
94/4133
95/4133
96/4133
97/4133
98/4133
99/4133
100/4133
101/4133
102/4133
103/4133
104/4133
105/4133
106/4133
107/4133
108/4133
109/4133
110/4133
111/4133
112/4133
113/4133
114/4133
115/4133
116/4133
117/4133
118/4133
119/4133
120/4133
121/4133
122/4133
123/4133
1

UnicodeEncodeError: 'cp949' codec can't encode character '\U0010065b' in position 61: illegal multibyte sequence

In [ ]:
table

<table>
<colgroup>
<col width="78px"/>
<col width="*"/>
<col width="80px"/>
<col width="140px"/>
<col width="125px"/>
<col width="135px"/>
<col width="65px"/>
<col width="42px"/>
<col width="65px"/>
</colgroup>
<thead>
<tr>
<th class="first"><a href="javascript:_Order_Link(10000000,1000000,'/analysis/list?&amp;sdate=2000-01-01&amp;edate=2009-12-31&amp;report_type=CO&amp;order_type=&amp;now_page=4132')">작성일 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th><a href="javascript:_Order_Link(10000000,10000,'/analysis/list?&amp;sdate=2000-01-01&amp;edate=2009-12-31&amp;report_type=CO&amp;order_type=&amp;now_page=4132')">제목 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th><a href="javascript:_Order_Link(10000000,10,'/analysis/list?&amp;sdate=2000-01-01&amp;edate=2009-12-31&amp;report_type=CO&amp;order_type=&amp;now_page=4132')">적정가격 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th><a href="javascript:_Order_Link(10000000,1,'/analysis/list?&amp;sdate=2000-01-01&amp;edate=20

In [ ]:
data

,작성일,종목명,종목코드,제목,적정가격,평가의견,작성자,작성기관
0,2009-12-30,신세계,004170,에 대해 매수의견과 적정주가 62만원을 유지한다 10년 이마트 매출 턴어 라운드 ...,"620,000",매수,여영상,신한금융
1,2009-12-30,KT,030200,약 5 에 달하는 높은 배당수익률과 자사주 매입 등의 우호적인 주주환원 정책을 고...,"48,400",매수,진창환,신한금융
2,2009-12-30,기아,000270,차 차 차의 신차 판매비율과 원가율간의 개연성은 뉴 모닝이 출시된 2008년부터 ...,"23,000",매수,이기정,신한금융
3,2009-12-30,LG전자,066570,LG 전자 LG 전자 취약한 스마트폰과 LED TV 경쟁력으로 인해 2009년 4...,"143,000",매수,소현철,신한금융
4,2009-12-30,삼성전자,005930,에 대해 투자의견 매수 목표주가 95만원을 유지한다 최근 3개월간 주 가는 비수기...,"950,000",매수,진성혜,신한금융
...,...,...,...,...,...,...,...,...
31825,2003-10-07,삼일제약,000520,3분기 매출액은 6 9 증가한 것으로 추정됨 이는 부루펜 콜디 등 소아과용약의 매...,"68,000",적극매수,김지현,동원증권
31826,2003-10-07,SK텔레콤,017670,매수 매수 6개월목표주가 258000원 수익성 호전불구 과매도괸 경기방어주,"258,000",매수,양종인ㆍ조성옥,동원증권
31827,2003-10-07,LG전자,066570,Market Performer Market Performer 6개월 목표주가 66...,"66,000",MKTPER,"임홍빈,임정민",삼성증권
31828,2003-10-07,삼성전자,005930,BUY BUY 6개월 목표주가 495 000원 보통주 목표주가 유지 이유 1 의 ...,"495,000",매수,임홍빈,삼성증권


In [ ]:

data.to_csv('/content/gdrive/MyDrive/Colab Notebooks/리포트_데이터_~2010.csv', index=False)
